# Introduction

In Q-leraning, we maintain a matrix to save all the action values. However, this does not work for continuous space, or a complex environment with huge action and state spaces. It is natural to consider replace the Q-table by a function. In other words, we use a function to approximate Q. This is the core idea of DQN (Deep-Q Network).

Notice that DQN may not be suitable for continuous action space, since we will do $\max Q$ for the value updates, which could be hard for an NN if the actions are not discrete. 

First we write out the update rulws for Q-learning: $$Q(s, a) \leftarrow Q(s, a) + \alpha (r + \gamma \max_{a'}Q(s', a')- Q(s, a))$$

# CartPole env
In the CartPole Env, there exists a cart, on top of which stands a pole. The agent is required to move the cart horizontally such to make the pole standing. If the pole tilts too much, or the cart moves too far from the initial place, or neither of the two happenes for 200 frames, the game ends. The state of the agent is a tuple of (cart_position, cart_velocity, pole_angle, pole_tip_velocity). We have 2 actions, 0 for left and 1 for right.

# DQN for CartPole env

Instead of learning a function $f: (s, a) \rightarrow Q$, we can simply set it to learn the action given the state, i.e. $f: s \rightarrow a$. 

Now an improtant question comes to us: how do we define the loss function?

It is simple to use an MSE loss between $Q_\omega(s, a)$ (the learnt action value) and the TD estimation $r + \gamma \max_{a'}Q(s', a')$, which theoretically should equal the actaul action values:

$$\omega^* = \arg \min_{\omega} \frac{1}{2N}\sum_{i=1}^N[Q_\omega(s_i, a_i) - (r + \gamma \max_{a'}Q(s'_i, a'))]^2$$

Now that we have the loss function, we have expand RL into its NN form. Since DQN use the same idea as of Q-learning, it is also off-policy. Therefore, we can banalnce exploration and exploitation by $\epsilon$-greedy, and collect the sampled data for later updates. 

Before we implement DQN, there are 2 modules we need to know that facilitates DQN training to be stable and eminent, namely experience replay and target network. 

## Experience replay

Consider a supervised learning task, where we sample 1 or a batch of data for 1 gradient update. With the training goes on, especially when we have multiple epochs, a certain data sample is used multiple times. The reason why we can do this is that in supervised learning, we have an important assumption: the i.i.d assumption that says, all the data in the training set are independent from an identical distrituion. This is also the reason why RL, involving temporal data, could be hard to train using supervised methods: because the temporal struction breaks the i.i.d assumption. 

Since we use an NN to estimate $Q$, we will need much more data to feed it so that the network is thoroughly trained. This is why we need experience replay. 

In experience replay, we maintain a replay-loading area, where you will find all the sampled 4-element tuples $(s, a, r, s')$. In the training of the Q-network, we randomly sample from the area. By doing so, the training data satisfies i.i.d assumption, and each data sample can be used multiple times to train the network. 